In [23]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

#docs = ['why hello there', 'omg hello pony', 'she went there? omg']
def termDocumentMatrix(docs):
    vec = CountVectorizer()
    X = vec.fit_transform(docs)
    df = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())
    return df

In [24]:
aa = ['John and Bob are brothers.','John went to the store. The store was closed.','Bob went to the store too.']
termDocumentMatrix(aa)

,and,are,bob,brothers,closed,john,store,the,to,too,was,went
0,1,1,1,1,0,1,0,0,0,0,0,0
1,0,0,0,0,1,1,2,2,1,0,1,1
2,0,0,1,0,0,0,1,1,1,1,0,1


### Document Term Matrix with TF-IDF

In [9]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
 
def termMatrix(doc):
    vectorizer = TfidfVectorizer()
    doc_vec = vectorizer.fit_transform(doc)
    df2 = pd.DataFrame(doc_vec.toarray().transpose(),
                   index=vectorizer.get_feature_names())
    return df2

In [10]:
aa = ['John and Bob are brothers.','John went to the store. The store was closed.','Bob went to the store too.']
termMatrix(aa)

,0,1,2
and,0.490479,0.000000,0.000000
are,0.490479,0.000000,0.000000
bob,0.373022,0.000000,0.385503
brothers,0.490479,0.000000,0.000000
closed,0.000000,0.345808,0.000000
john,0.373022,0.262996,0.000000
store,0.000000,0.525991,0.385503
the,0.000000,0.525991,0.385503
to,0.000000,0.262996,0.385503
too,0.000000,0.000000,0.506890


# Permuterm Index

In [12]:
import os

with open('Stopwords.txt') as f:
    for line in f:
        stopwords = line.split(", ")
#print stopwords

tokens = {}
documentID = 0
path = r"C:\Users\Phu Wai Paing\Desktop\Course Books\Information Retrieval\dataExtraction\dataset"

for root, dirs, files in os.walk(path):
    for file in files:
        with open(os.path.join(path, file)) as f:
                documentID += 1
                line_tokens = []
                for line in f:
                    line_tokens = line.split(  )
                    for each in line_tokens:
                        if each not in stopwords:
                            if each not in tokens:
                                tokens[each] = [documentID]
                            else:
                                tokens[each].append(documentID)

file = open("InvertedIndex.txt", "w")
for key in sorted(tokens):
    file.write(key)
    file.write(" ")
    value = ','.join(str(v) for v in tokens[key])
    file.write(value)
    file.write("\n")
file.close()

def rotate(str, n):
    return str[n:] + str[:n]

file = open("PermutermIndex.txt","w")
keys = tokens.keys()
for key in sorted(keys):
    dkey = key + "$"
    for i in range(len(dkey),0,-1):
        out = rotate(dkey,i)
        file.write(out)
        file.write(" ")
        file.write(key)
        file.write("\n")
file.close()

In [14]:
rotate("hello",4)

'ohell'

In [17]:
import os

query = input('Enter Query : ')

parts = query.split("*")
print(len(parts))

#    1. X*               -->         X 
#	2. *X				-->			X$*
#	3. X*Y				-->			Y$X*
#	4. X*Y*Z			-->			(Z$X*) and (Y*)
#	5. *X* 				-->			can be converted to X* form
    
if len(parts) == 3:
    case = 4
elif parts[1] == '':
    case = 1
elif parts[0] == '':
    case = 2
elif parts[0] != '' and parts[1] != '':
    case = 3

if case == 4:
    if parts[0] == '':
        case = 1

print("case = ", case)

inverted = {}
with open(r'C:\Users\Phu Wai Paing\Desktop\Course Books\Information Retrieval\dataExtraction\InvertedIndex.txt') as f:
    for line in f:
        temp = line.split( )
        val = temp[1].split(",")
        inverted[temp[0]] = val

print(inverted)    

permuterm = {}
with open(r'C:\Users\Phu Wai Paing\Desktop\Course Books\Information Retrieval\dataExtraction\PermutermIndex.txt') as f:
    for line in f:
        temp = line.split( )
        permuterm[temp[0]] = temp[1]
        
print(permuterm)

def prefix_match(term, prefix):
    term_list = []
    for tk in term.keys():
        if tk.startswith(prefix):
            term_list.append(term[tk])
    return term_list
        
docID = 0

def processQuery(query):    
    term_list = prefix_match(permuterm,query)
    print(term_list)
    
    docID = []
    for term in term_list:
        docID.append(inverted[term])
    print(docID)

    temp = []
    for x in docID:
        for y in x:
            temp.append(y)
    print(temp)        

    temp = [int(x) for x in temp]
    documentID = 0
    outputfile = open("RetrievedDocuments.txt","w")
    path = r"C:\Users\Phu Wai Paing\Desktop\Course Books\Information Retrieval\dataExtraction\dataset"
    for root, dirs, files in os.walk(path):
        for file in files:
            documentID = documentID + 1
            with open(os.path.join(path, file)) as f:
                for text in f:
                    if documentID in temp:
                        outputfile.write(file + "\n" + text + "\n")
            f.close()
    outputfile.close()
    return 

#    1. X*               -->         X 
#	2. *X				-->			X$*
#	3. X*Y				-->			Y$X*
#	4. X*Y*Z			-->			(Z$X*) and (Y*)
#	5. *X* 				-->			can be converted to X* form

if case == 1:
    query = parts[0]
elif case == 2:
    query = parts[1] + "$"
elif case == 3:
    query = parts[1] + "$" + parts[0]
elif case == 4:
    queryA = parts[2] + "$" + parts[0]
    queryB = parts[1]

def bitwise_and(A,B):
    return set(A).intersection(B)
    
if case != 4:
    processQuery(query)
elif case == 4:
# queryA Z$X
    term_list = prefix_match(permuterm,queryA)
    #print(term_list)
    
    docID = []
    for term in term_list:
        docID.append(inverted[term])
    #print(docID)

    temp1 = []
    for x in docID:
        for y in x:
            temp1.append(y)
    #print(temp)        

    temp1 = [int(x) for x in temp1]
# queryB Y
    term_list = prefix_match(permuterm,queryB)
    #print(term_list)
    
    docID = []
    for term2 in term_list:
        docID.append(inverted[term2])
    #print(docID)

    temp2 = []
    for x in docID:
        for y in x:
            temp2.append(y)
    #print(temp)        

    temp2 = [int(x) for x in temp2]

    temp = bitwise_and(temp1,temp2)

  #  print(temp1,temp2,temp)    
    documentID = 0
    outputfile = open("RetrievedDocuments.txt","w")
    path = r"C:\Users\Phu Wai Paing\Desktop\Course Books\Information Retrieval\dataExtraction\dataset"
    for root, dirs, files in os.walk(path):
        for file in files:
            documentID = documentID + 1
            with open(os.path.join(path, file)) as f:
                for text in f:
                    if documentID in temp:
                        outputfile.write(file + "\n" + text + "\n")
            f.close()
    outputfile.close()

Enter Query : f*e
2
case =  3
{'big': ['1'], 'bubble': ['3'], 'burst': ['3'], 'data': ['1', '2', '3'], 'future': ['1'], 'linkedin': ['3'], 'mining': ['3'], 'new': ['2'], 'oil': ['2'], 'predicts': ['3'], 'statistics': ['3']}
{'big$': 'big', '$big': 'big', 'g$bi': 'big', 'ig$b': 'big', 'bubble$': 'bubble', '$bubble': 'bubble', 'e$bubbl': 'bubble', 'le$bubb': 'bubble', 'ble$bub': 'bubble', 'bble$bu': 'bubble', 'ubble$b': 'bubble', 'burst$': 'burst', '$burst': 'burst', 't$burs': 'burst', 'st$bur': 'burst', 'rst$bu': 'burst', 'urst$b': 'burst', 'data$': 'data', '$data': 'data', 'a$dat': 'data', 'ta$da': 'data', 'ata$d': 'data', 'future$': 'future', '$future': 'future', 'e$futur': 'future', 're$futu': 'future', 'ure$fut': 'future', 'ture$fu': 'future', 'uture$f': 'future', 'linkedin$': 'linkedin', '$linkedin': 'linkedin', 'n$linkedi': 'linkedin', 'in$linked': 'linkedin', 'din$linke': 'linkedin', 'edin$link': 'linkedin', 'kedin$lin': 'linkedin', 'nkedin$li': 'linkedin', 'inkedin$l': 'linkedin

# Soundex

In [40]:
from itertools import groupby
 
def soundex(word):
   codes = ("bfpv","cgjkqsxz", "dt", "l", "mn", "r")
   soundDict = dict((ch, str(ix+1)) for ix,cod in enumerate(codes) for ch in cod)
   cmap2 = lambda a: soundDict.get(a, '9')
   sdx =  ''.join(cmap2(a) for a in word.lower())
   sdx2 = word[0].upper() + ''.join(k for k,g in list(groupby(sdx))[1:] if k!='9')
   sdx3 = sdx2[0:4].ljust(4,'0')
   return sdx3

In [46]:
soundex("anbmgrams")

'A515'

# SPIMI_Invert

In [81]:
import sys
import ast
import re
import collections

from collections import OrderedDict

def spimi_invert(documents, block_size_limit):
    """ Applies the Single-pass in-memory indexing algorithm """
    block_number = 0
    documents_count = len(documents)
    dictionary = {} # (term - postings list)
    for index, docID in enumerate(documents):
        for term in documents[docID]:
            # If term occurs for the first time
            if term not in dictionary:
                # Add term to dictionary, create new postings list, and add docID
                dictionary[term] = [docID]
            # else:
            #     # If term has a subsequent occurence
            #     if docID not in dictionary[term]:
            #         # Add a posting (docID) to the existing posting list of the term
            #         dictionary[term].append(docID)
            else:
                dictionary[term].append(docID)
        if sys.getsizeof(dictionary) > block_size_limit or (index == documents_count-1):
            temp_dict = sort_terms(dictionary)
            write_block_to_disk(temp_dict, block_number)
            temp_dict = {}
            block_number += 1
            dictionary = {}
    print("SPIMI invert complete!")

def sort_terms(term_postings_list):
    """ Sorts dictionary terms in alphabetical order """
    print(" -- Sorting terms...")
    sorted_dictionary = OrderedDict() # keep track of insertion order
    sorted_terms = sorted(term_postings_list)
    for term in sorted_terms:
        result = [int(docIds) for docIds in term_postings_list[term]]
        result_tftd = calculate_tftd(result)
        sorted_dictionary[term] = result_tftd
    return sorted_dictionary

def calculate_tftd(pl_with_duplicates):
    """ Add term frequency of term in each document """
    # print(pl_with_duplicates)
    counter = collections.Counter(pl_with_duplicates)
    pl_tftd = [[int(docId), counter[docId]] for docId in counter.keys()]
    return pl_tftd

def write_block_to_disk(term_postings_list, block_number):
    """ Writes index of the block (dictionary + postings list) to disk """
    # Define block
    base_path = 'index_blocks/'
    block_name = 'block-' + str(block_number) + '.txt'
    block = open(base_path + block_name, 'a+')
    print(" -- Writing term-positing list block: " + block_name + "...")
    # Write term : posting lists to block
    for index, term in enumerate(term_postings_list):
        # Term - Posting List Format
        # term:[docID1, docID2, docID3]
        # e.g. cat:[4,9,21,42]
        block.write(str(term) + ":" + str((term_postings_list[term])) + "\n")
    block.close()